<a href="https://colab.research.google.com/github/JoshOdegai/Josh_DTSC3020_Fall2025/blob/main/Assignment_6_WebScraping_.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Assignment 6 (4 points) — Web Scraping

In this assignment you will complete **two questions**. The **deadline is posted on Canvas**.


## Assignment Guide (Read Me First)

- This notebook provides an **Install Required Libraries** cell and a **Common Imports & Polite Headers** cell. Run them first.
- Each question includes a **skeleton**. The skeleton is **not** a solution; it is a lightweight scaffold you may reuse.
- Under each skeleton you will find a **“Write your answer here”** code cell. Implement your scraping, cleaning, and saving logic there.
- When your code is complete, run the **Runner** cell to print a Top‑15 preview and save the CSV.
- Expected outputs:
  - **Q1:** `data_q1.csv` + Top‑15 sorted by the specified numeric column.
  - **Q2:** `data_q2.csv` + Top‑15 sorted by `points`.


In [1]:
# 1) Install Required Libraries
!pip -q install requests beautifulsoup4 lxml pandas
print("Dependencies installed.")


Dependencies installed.


### 2) Common Imports & Polite Headers

In [2]:
# Common Imports & Polite Headers
import re, sys, pandas as pd, requests
from bs4 import BeautifulSoup
HEADERS = {"User-Agent": (
    "Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 "
    "(KHTML, like Gecko) Chrome/122.0 Safari/537.36")}
def fetch_html(url: str, timeout: int = 20) -> str:
    r = requests.get(url, headers=HEADERS, timeout=timeout)
    r.raise_for_status()
    return r.text
def flatten_headers(df: pd.DataFrame) -> pd.DataFrame:
    if isinstance(df.columns, pd.MultiIndex):
        df.columns = [" ".join([str(x) for x in tup if str(x)!="nan"]).strip()
                      for tup in df.columns.values]
    else:
        df.columns = [str(c).strip() for c in df.columns]
    return df
print("Common helpers loaded.")


Common helpers loaded.


## Question 1 — IBAN Country Codes (table)
**URL:** https://www.iban.com/country-codes  
**Extract at least:** `Country`, `Alpha-2`, `Alpha-3`, `Numeric` (≥4 cols; you may add more)  
**Clean:** trim spaces; `Alpha-2/Alpha-3` → **UPPERCASE**; `Numeric` → **int** (nullable OK)  
**Output:** write **`data_q1.csv`** and **print a Top-15** sorted by `Numeric` (desc, no charts)  
**Deliverables:** notebook + `data_q1.csv` + short `README.md` (URL, steps, 1 limitation)

**Tip:** You can use `pandas.read_html(html)` to read tables and then pick one with ≥3 columns.


In [12]:
# --- Q1 Skeleton (fill the TODOs) ---
def q1_read_table(html: str) -> pd.DataFrame:
    """Return the first table with >= 3 columns from the HTML.
    TODO: implement with pd.read_html(html), pick a reasonable table, then flatten headers.
    """
    tables = pd.read_html(html)
    for table in tables:
      if table.shape[1] >= 3:
        return flatten_headers(table)
    raise NotImplementedError("TODO: implement q1_read_table")

def q1_clean(df: pd.DataFrame) -> pd.DataFrame:
    """Clean columns: strip, UPPER Alpha-2/Alpha-3, cast Numeric to int (nullable), drop invalids.
    TODO: implement cleaning steps.
    """
    cols = {c.lower(): c for c in df.columns}  # case-insensitive
    country_col = next((cols[k] for k in cols if "country" in k), None)
    alpha2_col = next((cols[k] for k in cols if "alpha-2" in k or "alpha 2" in k), None)
    alpha3_col = next((cols[k] for k in cols if "alpha-3" in k or "alpha 3" in k), None)
    numeric_col = next((cols[k] for k in cols if "numeric" in k), None)

    # Filter to needed columns (ignore extras if present)
    keep = [c for c in [country_col, alpha2_col, alpha3_col, numeric_col] if c]
    df = df[keep].copy()

    # Normalized names
    new_names = ["Country", "Alpha-2", "Alpha-3", "Numeric"]
    df.columns = new_names[:len(df.columns)]

    # Strip string whitespace where applicable
    for col in ["Country", "Alpha-2", "Alpha-3"]:
        if col in df.columns:
            df[col] = df[col].astype(str).str.strip()

    # Uppercase ISO codes
    if "Alpha-2" in df.columns:
        df["Alpha-2"] = df["Alpha-2"].str.upper()
    if "Alpha-3" in df.columns:
        df["Alpha-3"] = df["Alpha-3"].str.upper()

    # Convert Numeric → Int64 (nullable)
    if "Numeric" in df.columns:
        df["Numeric"] = df["Numeric"].astype("Int64")

    # Drop rows missing the two most essential fields
    df = df.dropna(subset=["Country", "Alpha-2"])

    return df.reset_index(drop=True)
    raise NotImplementedError("TODO: implement q1_clean")

def q1_sort_top(df: pd.DataFrame, top: int = 15) -> pd.DataFrame:
    """Sort descending by Numeric and return Top-N.
    TODO: implement.
    """
    df_sorted = df.dropna(subset=["Numeric"]).sort_values(
      by="Numeric", ascending=False
    )
    return df_sorted.head(top).reset_index(drop=True)
    raise NotImplementedError("TODO: implement q1_sort_top")


In [13]:
# Q1 — Write your answer here
URL = "https://www.iban.com/country-codes"


html = fetch_html(URL)
df_raw = q1_read_table(html)
df_clean = q1_clean(df_raw)

df_clean.to_csv("data_q1.csv", index=False)

print(q1_sort_top(df_clean, top=15))





                                              Country Alpha-2 Alpha-3  Numeric
0                                              Zambia      ZM     ZMB      894
1                                               Yemen      YE     YEM      887
2                                               Samoa      WS     WSM      882
3                                   Wallis and Futuna      WF     WLF      876
4                  Venezuela (Bolivarian Republic of)      VE     VEN      862
5                                          Uzbekistan      UZ     UZB      860
6                                             Uruguay      UY     URY      858
7                                        Burkina Faso      BF     BFA      854
8                               Virgin Islands (U.S.)      VI     VIR      850
9                      United States of America (the)      US     USA      840
10                       Tanzania, United Republic of      TZ     TZA      834
11                                        Isle of Ma

/tmp/ipython-input-3383860988.py:6: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  tables = pd.read_html(html)


## Question 2 — Hacker News (front page)
**URL:** https://news.ycombinator.com/  
**Extract at least:** `rank`, `title`, `link`, `points`, `comments` (user optional)  
**Clean:** cast `points`/`comments`/`rank` → **int** (non-digits → 0), fill missing text fields  
**Output:** write **`data_q2.csv`** and **print a Top-15** sorted by `points` (desc, no charts)  
**Tip:** Each story is a `.athing` row; details (points/comments/user) are in the next `<tr>` with `.subtext`.


In [9]:
# --- Q2 Skeleton (fill the TODOs) ---
def q2_parse_items(html: str) -> pd.DataFrame:
    """Parse front page items into DataFrame columns:
       rank, title, link, points, comments, user (optional).
    TODO: implement with BeautifulSoup on '.athing' and its sibling '.subtext'.
    """
    soup = BeautifulSoup(html, "html.parser")
    items = []

    rows = soup.select("tr.athing")
    for row in rows:
        rank_tag = row.select_one(".rank")

        title_tag = row.select_one(".titleline a")
        link = title_tag["href"] if title_tag and title_tag.has_attr("href") else ""

        sub = row.find_next_sibling("tr")
        subtext = sub.select_one(".subtext") if sub else None

        points_tag = subtext.select_one(".score") if subtext else None
        user_tag = subtext.select_one(".hnuser") if subtext else None

        comments_tag = None
        if subtext:
            links = subtext.find_all("a")
            if links:
                comments_tag = links[-1]

        items.append({
            "rank": rank_tag.text.strip().replace(".", "") if rank_tag else "",
            "title": title_tag.text.strip() if title_tag else "",
            "link": link,
            "points": points_tag.text.strip() if points_tag else "",
            "comments": comments_tag.text.strip() if comments_tag else "",
            "user": user_tag.text.strip() if user_tag else ""
        })

    return pd.DataFrame(items)
    raise NotImplementedError("TODO: implement q2_parse_items")

def q2_clean(df: pd.DataFrame) -> pd.DataFrame:
    """Clean numeric fields and fill missing values.
    TODO: cast points/comments/rank to int (non-digits -> 0). Fill text fields.
    """
    df = df.copy()

    for col in ["rank", "points", "comments"]:
        df[col] = (
            df[col]
            .astype(str)
            .str.extract(r"(\d+)", expand=False)
            .fillna("0")
            .astype(int)
        )

    for col in ["title", "link", "user"]:
        if col in df.columns:
            df[col] = df[col].fillna("").astype(str).str.strip()

    return df.reset_index(drop=True)
    raise NotImplementedError("TODO: implement q2_clean")

def q2_sort_top(df: pd.DataFrame, top: int = 15) -> pd.DataFrame:
    """Sort by points desc and return Top-N. TODO: implement."""
    return df.sort_values(by="points", ascending=False) \
             .head(top).reset_index(drop=True)
    raise NotImplementedError("TODO: implement q2_sort_top")


In [10]:
# Q2 — Write your answer here
URL = "https://news.ycombinator.com/"

html = fetch_html(URL)
df_raw = q2_parse_items(html)
df_clean = q2_clean(df_raw)

df_clean.to_csv("data_q2.csv", index=False)

print(q2_sort_top(df_clean, top=15))



    rank                                              title  \
0     18                   Solarpunk is happening in Africa   
1     29                          End of Japanese community   
2     11                             Ratatui – App Showcase   
3     24      Dillo, a multi-platform graphical web browser   
4     28  ChatGPT terms disallow its use in providing le...   
5     27  Firefox profiles: Private, focused spaces for ...   
6      7            FBI tries to unmask owner of archive.is   
7      5  Open Source Implementation of Apple's Private ...   
8      3  Kimi K2 Thinking, a SOTA open-source trillion-...   
9     17  Cloudflare Tells U.S. Govt That Foreign Site B...   
10     1             ICC ditches Microsoft 365 for openDesk   
11    14    Mathematical exploration and discovery at scale   
12    23  I may have found a way to spot U.S. at-sea str...   
13    22                  How I am deeply integrating Emacs   
14    12  Australia has so much solar that it's offerin